![PyMC Labs](images/4-pymc-labs-transp-black.png)

# Session 3: Markov Chain Monte Carlo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az
import scipy.stats as st
import platform
import warnings
warnings.simplefilter("ignore")

plt.rcParams['figure.dpi'] = 75

if platform.system() == 'Linux':
    # Multiprocessing behaves oddly on Linux, so we need to set the start method
    import multiprocessing as mp
    mp.set_start_method('spawn', force=True)

RANDOM_SEED = 20090425

## Bayesian Computation

Let's take a look at [Bayes formula](https://en.wikipedia.org/wiki/Bayes%27_theorem):

$$P(\theta|x) = \frac{P(x|\theta) P(\theta)}{P(x)}$$

We have $P(\theta|x)$, the probability of our model parameters $\theta$ given the data $x$ and thus our quantity of interest. To compute this we multiply the prior $P(\theta)$ (what we think about $\theta$ before we have seen any data) and the likelihood $P(x|\theta)$, i.e. how we think our data is distributed. This nominator is pretty easy to solve for.

However, let's take a closer look at the denominator. $P(x)$ which is also called the evidence (i.e. the evidence that the data x was generated by this model). We can compute this quantity by integrating over all possible parameter values:
$$P(x) = \int_\Theta P(x, \theta) \, \mathrm{d}\theta$$

This is the key difficulty with Bayes formula -- while the formula looks innocent enough, for even slightly non-trivial models you just can't compute the posterior in a closed-form way. 

## Numerical Integration

Bayesian analysis often requires integration over multiple dimensions that is intractable both via analytic methods or standard methods of numerical integration.
However, it is often possible to compute these integrals by simulating
(drawing samples) from posterior distributions. For example, consider the expected value of a random variable $\mathbf{x}$:

$$E[\mathbf{x}] = \int \mathbf{x} f(\mathbf{x}) d\mathbf{x}, \qquad\mathbf{x} = x_1, \ldots ,x_k$$

where $k$ (the dimension of vector $x$) is perhaps very large. If we can produce a reasonable number of random vectors ${\bf x_i}$, we can use these values to approximate the unknown integral. This process is known as *Monte Carlo integration*. In general, MC integration allows integrals against probability density functions:

$$I = \int h(\mathbf{x}) f(\mathbf{x}) \mathbf{dx}$$

to be estimated by finite sums:

$$\hat{I} = \frac{1}{n}\sum_{i=1}^n h(\mathbf{x}_i),$$

where $\mathbf{x}_i$ is a sample from $f$. This estimate is valid and useful because:

-   By the strong law of large numbers:

$$\hat{I} \rightarrow I   \text{   with probability 1}$$

-   Simulation error can be measured and controlled:

$$Var(\hat{I}) = \frac{1}{n(n-1)}\sum_{i=1}^n (h(\mathbf{x}_i)-\hat{I})^2$$

## Rejection Sampling

Most integrals are hard or impossible to do. Also, if we are iterating on a statistical model, we may want a method that works without requiring rederiving a formula for generating samples. Further, in Bayesian data analysis, we may not know a *normalizing constant*: we may only know 

$$
\tilde{p}(x) = \frac{1}{Z_p}p(x),
$$

for some constant $Z_p$ ("constant" here is with respect to $x$). In order to sample, first we

1. Choose a proposal distribution $q$ that you know how to sample from
2. Choose a number $k$, so that $kq(x) \geq \tilde{p}(x)$ for all $x$

Then, we repeatedly 

1. Draw a $z$ from $q$
2. Draw a $u$ from $\operatorname{Uniform}(0, kq(z))$
3. If $u \leq p(x)$, accept the draw, otherwise, reject.

Importantly, every "rejection" is wasted computation! We will explore methods for having less wasted computation later.

### Example: Mixture of Gaussians

We can sample from the pdf returned by `mixture_of_gaussians` using rejection sampling. We will implement this as a Python generator, and yield the proposed draw, `z`, as well as whether it was accepted.

In [ ]:
import scipy.stats as st

def mixture_of_gaussians():
    rvs = (st.norm(-3, 1), st.norm(0, 1), st.norm(3, 1))
    probs = (0.5, 0.2, 0.3)
    def pdf(x):
        return sum(p * rv.pdf(x) for p, rv in zip(probs, rvs))
    return pdf

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(6)
pdf = mixture_of_gaussians()
q = st.norm(0, 3)
z = q.rvs()
u = np.random.rand() * q.pdf(z)
k = 3

fig, ax = plt.subplots(figsize=(10, 5), constrained_layout=True)

t = np.linspace(-10, 10, 500)
ax.plot(t, pdf(t), '-', label='$p(x)$')
ax.fill_between(t, 0, pdf(t), alpha=0.2)
ax.plot(t, k * q.pdf(t), '-', label='$k \cdot \mathcal{N}(z | 0, 3)$')
ax.fill_between(t, pdf(t), k * q.pdf(t), alpha=0.2)

bg_color = ax.get_facecolor()
ax.vlines(z, 0, pdf(z), linestyles='dashed', color='green')
ax.vlines(z, pdf(z), k * q.pdf(z), linestyles='dashed', color='red')

ax.plot(z, pdf(z), 'o', color='C0', ms=15, mfc=bg_color)
ax.plot(z, u, 'rx', label='$u \sim U(0, k\cdot\mathcal{N}(z | 0, 3))$', ms=15, mfc=bg_color)
ax.plot(z, k * q.pdf(z), 'o', color='C1', ms=15, mfc=bg_color)

ax.set_ylim(bottom=0)
ax.set_xlim(t.min(), t.max())
ax.legend()

> #### What is a Generator?
> A Python generator is a special type of iterator that allows you to iterate over a sequence of values lazily, meaning it generates values on-the-fly and only when requested, rather than storing the entire sequence in memory. This is accomplished using the `yield` statement within a function, which produces a value and pauses the function's execution, resuming from that point when the next value is requested. Thus, instead of generating and storing all values at once, which can be resource-intensive, generators produce values one at a time, thus reducing memory overhead and allowing for the handling of infinite sequences or very large data sets that would be impractical to store in memory. 

In [ ]:
# Rejection sampler code here
def rejection_sampler(pdf, proposal_dist, k):
    while True:
        # Generate proposal
        z = proposal_dist.rvs()
        q_dist = proposal_dist.pdf(z)
        # Draw uniformly from 0 to k * q(z)
        u = k * np.random.rand() * q_dist
        # Check our envelope condition
        assert k * q_dist >= pdf(z)
        if u <= pdf(z):
            accept = True
        else:
            accept = False
        yield z, accept

def gen_samples(draws, sampler):
    samples = []
    for n_draws, (z, accept) in enumerate(sampler, start=1):
        if accept:
            samples.append(z)
            if len(samples) == draws:
                return np.array(samples), n_draws

In [ ]:
pdf = mixture_of_gaussians()
proposal_dist = st.norm(0, 3)
k = 4
N = 10_000

samples, draws = gen_samples(N, rejection_sampler(pdf, proposal_dist, k))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

t = np.linspace(samples.min(), samples.max(), 500)

# This histogram should look very similar to the pdf that is plotted
ax.hist(samples, bins='auto', density=True)
ax.plot(t, pdf(t))

ax.set_title(f'{samples.size:,d} draws from the pdf with {100 * samples.size / draws:.2f}% efficiency');

## Introduction to MCMC

One way to intuitively waste less computation is to use knowledge from your current sample to inform your next proposal: this is called a *Markov chain*. 


> ## Markov Chains
>
> A Markov chain is a special type of *stochastic process*. The standard definition of a stochastic process is an ordered collection of random variables:
>
> $$\{X_t: t \in T\}$$
> 
> where $t$ is frequently (but not necessarily) a time index. If we think of $X_t$ as a state $X$ at time $t$, and invoke the following dependence condition on each state:
> 
> $$Pr(X_{t+1}=x_{t+1} | X_t=x_t, X_{t-1}=x_{t-1},\ldots,X_0=x_0) = Pr(X_{t+1}=x_{t+1} | X_t=x_t)$$
> 
> then the stochastic process is known as a Markov chain. This conditioning specifies that the future depends on the current state, but not past states.


Let $t$ be the index of our current sample, $x_t$ be our current sample, and $\operatorname{pdf}(x_t)$ be our probability density function evaluated at the current sample. We will define a *transition probability* that is conditioned on our current position: $T(x_{t + 1} | x_t)$. It turns out that a Markov chain will sample from $\operatorname{pdf}$ if:

- $T$ is ergodic (sort of techinical -- roughly $T$ is aperiodic and can explore the whole space)
- The chain satisfies *detailed balance*, which means $\operatorname{pdf}(x_t)T(x_{t+1} | x_t) = \operatorname{pdf}(x_{t + 1})T(x_{t} | x_{t + 1})$.

This second criteria inspires the *Metropolis acceptance criteria*: If we use any proposal with density function $\operatorname{prop}$, we use this criterion to "correct" the transition probability to satisfy detailed balance:

$$
A(x_{t + 1} | x_t) = \min\left\{1, \frac{\operatorname{pdf}(x_{t + 1})}{\operatorname{pdf}(x_{t})}\frac{\operatorname{prop}(x_{t} | x_{t + 1})}{\operatorname{prop}(x_{t + 1} | x_t)} \right\}
$$

Now the *Metropolis-Hastings Algorithm* is

Initialize at some point $x_0$. For each iteration:

1. Draw $\tilde{x}_{t + 1} \sim \operatorname{prop}(x_t)$
2. Draw $u \sim \operatorname{Uniform}(0, 1)$
3. If $u < A(\tilde{x}_{t + 1} | x_t)$, then $x_{t + 1} = \tilde{x}_{t + 1}$. Otherwise, $x_{t + 1} = x_t$.

In [ ]:
def metropolis_hastings(pdf, prop_dist, init=0):
    """
    Yields a sample, and whether it was accepted. Notice that,
    unlike the rejection sampler, even when the second argument is `False`,
    we use the sample! 
    """
    current = init
    while True:
        prop = prop_dist.rvs()
        p_accept = min(1, pdf(prop) / pdf(current) * prop_dist.pdf(current) / prop_dist.pdf(prop))
        accept = np.random.rand() < p_accept
        if accept:
            current = prop
        yield current, accept
        
def gen_samples(draws, sampler):
    """
    An example of using the metropolis_hastings API.
    """
    samples = np.empty(draws)
    accepts = 0
    for idx, (z, accept) in enumerate(sampler):
        if idx >= draws:
            break
        accepts += int(accept)
        samples[idx] = z
    return samples, accepts

In [ ]:
pdf = mixture_of_gaussians()
proposal_dist = st.norm(0, 10)

samples, accepts = gen_samples(10_000, metropolis_hastings(pdf, proposal_dist))

fig, ax = plt.subplots(figsize=(10, 7))

t = np.linspace(samples.min(), samples.max(), 500)
ax.hist(samples, bins='auto', density=True)
ax.plot(t, pdf(t))

ax.set_title(f'{samples.size:,d} draws from the pdf with {100 * accepts / samples.size :.2f}% accept rate');

### Random Walk Metropolis-Hastings

The implementation above is wildly inefficient! We will speed it up by fixing the proposal distribution as a Gaussian centered at the previous point (this is fairly standard). Specifically,
$$x_{t+1} \sim \mathcal{N}( x_t, \sigma),$$
so
$$\operatorname{prop}(x_{t+1} | x_{t}) = \mathcal{N}(x_{t + 1} | x_t, \sigma)$$

We call $\sigma$ the *step size*.

### Random Walk MCMC

A common special case of Metropolis-Hastings is random walk MCMC. In random walk MCMC, the proposal distribution is symmetric around the current position. This simplifies the acceptance ratio calculation because the ratio of the proposal distributions cancels out:

$$A(x_{t + 1} | x_t) = \min\left\{1, \frac{\operatorname{pdf}(x_{t + 1})}{\operatorname{pdf}(x_{t})}\right\}$$

This is simpler to implement and can be more efficient in some cases.

In [ ]:
def random_walk_metropolis(pdf, step_size, init=0):
    """Random walk Metropolis algorithm"""
    current = init
    while True:
        # Random walk proposal
        prop = current + np.random.normal(0, step_size)
        # Simple acceptance ratio (proposal terms cancel out)
        p_accept = min(1, pdf(prop) / pdf(current))
        accept = np.random.rand() < p_accept
        if accept:
            current = prop
        yield current, accept

In [ ]:
pdf = mixture_of_gaussians()

# Try different step sizes
for step_size in [0.1, 1.0, 5.0]:
    samples, accepts = gen_samples(5_000, random_walk_metropolis(pdf, step_size))
    
    fig, ax = plt.subplots(figsize=(10, 5))
    t = np.linspace(samples.min(), samples.max(), 500)
    ax.hist(samples, bins='auto', density=True)
    ax.plot(t, pdf(t))
    ax.set_title(f'Step size: {step_size}, Accept rate: {100 * accepts / samples.size:.2f}%');

## Gibbs Sampling

If you can sample from all the marginal distributions, you can implement a sampler pretty efficiently just using those.

Here is a stereotypical Gibbs sampling algorithm:

1.  Choose starting values for states (parameters):
    $\mathbf \theta = [\theta_1^{(0)},\theta_2^{(0)},\ldots,\theta_k^{(0)}]$

2.  Initialize counter $j=1$

3.  Draw the following values from each of the $k$ conditional
    distributions:

$$\begin{aligned}
\theta_1^{(j)} &\sim& \pi(\theta_1 | \theta_2^{(j-1)},\theta_3^{(j-1)},\ldots,\theta_{k-1}^{(j-1)},\theta_k^{(j-1)}) \\
\theta_2^{(j)} &\sim& \pi(\theta_2 | \theta_1^{(j)},\theta_3^{(j-1)},\ldots,\theta_{k-1}^{(j-1)},\theta_k^{(j-1)}) \\
\theta_3^{(j)} &\sim& \pi(\theta_3 | \theta_1^{(j)},\theta_2^{(j)},\ldots,\theta_{k-1}^{(j-1)},\theta_k^{(j-1)}) \\
\vdots \\
\theta_{k-1}^{(j)} &\sim& \pi(\theta_{k-1} | \theta_1^{(j)},\theta_2^{(j)},\ldots,\theta_{k-2}^{(j)},\theta_k^{(j-1)}) \\
\theta_k^{(j)} &\sim& \pi(\theta_k | \theta_1^{(j)},\theta_2^{(j)},\theta_4^{(j)},\ldots,\theta_{k-2}^{(j)},\theta_{k-1}^{(j)})
\end{aligned}$$

4.  Increment $j$ and repeat until convergence occurs.

This is pretty tricky to automate, since you need to know all of these conditional distributions! That said, this is often seen in science when a sampler is hand-built to do inference with a specific model. In that case, each conditional distribution might be computed by hand.

### Coal mining example
We have a time series of recorded coal mining disasters in the UK from 1851 to 1961.

Occurrences of disasters in the time series is thought to be derived from a Poisson process with a large rate parameter in the early part of the time series, and from one with a smaller rate in the later part. We are interested in locating the change point in the series, which perhaps is related to changes in mining safety regulations.

In [ ]:
disasters_array = np.array(
    [4, 5, 4, 0, 1, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6, 3, 3, 5, 4, 5, 3, 
     1, 4, 4, 1, 5, 5, 3, 4, 2, 5, 2, 2, 3, 4, 2, 1, 3, 2, 2, 1, 1, 1, 
     1, 3, 0, 0, 1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1, 0, 1, 
     0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2, 3, 3, 1, 1, 2, 1, 1, 1, 
     1, 2, 4, 2, 0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 
     1])
years = np.arange(1851, 1962, dtype=int)

fig, ax = plt.subplots()
ax.vlines(years, 0, disasters_array, lw=6)
ax.set_xlim(years.min() - 1, years.max() + 1)
ax.set_ylim(bottom=0);

### The Poisson changepoint model

Let's step through the construction of a model for this problem, starting with the likelihood.
It is natural to use a Poisson distribution for this type of count data. Denoting year $i$'s accident count by $y_i$, 

$$ y_i \sim \text{Poisson}(\lambda)  $$

The modeling problem revolves around estimating the values of the $\lambda$ parameters. Looking at the time series above, it appears that the rate declines later in the time series.

A ***changepoint model*** identifies a point (year) during the observation period (call it $\tau$) after which the parameter $\lambda$ drops to a lower value. So we are estimating two $\lambda$ parameters: one for the early period and another for the late period.

$$
\lambda = 
\begin{cases}
\lambda_1  & \text{if } t \lt \tau \cr
\lambda_2 & \text{if } t \ge \tau
\end{cases}
$$

We need to assign prior probabilities to both $\lambda$ parameters. The gamma distribution not only provides a continuous density function for positive numbers, but it is also **conjugate** with the Poisson sampling distribution. We will specify suitably vague hyperparameters $\alpha$ and $\beta$ for both priors.

$$\begin{aligned}
\lambda_1 &\sim \text{Gamma}( \alpha, \beta ) \cr
\lambda_2 &\sim \text{Gamma}( \alpha, \beta )
\end{aligned}$$

Since we do not have any intuition about the location of the changepoint (prior to viewing the data), we will assign a discrete uniform prior over all years 1851-1962.

$$\begin{aligned}
& \tau \sim \text{DiscreteUniform(1851,1962) }\cr
& \Rightarrow P( \tau = k ) = \frac{1}{111}
\end{aligned}$$

Now we can implement the model in PyMC:

In [ ]:
import pymc as pm

def coal_disaster_model():
    with pm.Model() as model:
        early_lambda = pm.Gamma('early_lambda', 1, 1)
        late_lambda = pm.Gamma('late_lambda', 1, 1)
        change_point = pm.DiscreteUniform('change_point', 1851, 1962)
        
        lam = pm.Deterministic('lam', pm.math.switch(years > change_point, late_lambda, early_lambda))
        pm.Poisson('rate', lam, observed=disasters_array)

    return model

pm.model_to_graphviz(coal_disaster_model())

### Implementing Gibbs sampling

We are interested in estimating the joint posterior of $\lambda_1$, $\lambda_2$ and $\tau$ given the array of annnual disaster counts $\mathbf{y}$. This gives:

$$
 P( \lambda_1, \lambda_2, \tau | \mathbf{y} ) \propto P(\mathbf{y} | \lambda_1, \lambda_2, \tau ) P(\lambda_1, \lambda_2, \tau) 
$$

To employ Gibbs sampling, we need to factor the joint posterior into the product of conditional expressions:

$$
 P( \lambda_1, \lambda_2, \tau | \mathbf{y} ) \propto P(y_{t<\tau} | \lambda_1, \tau) P(y_{t\ge \tau} | \lambda_2, \tau) P(\lambda_1) P(\lambda_2) P(\tau)
$$

which we have specified as:

$$\begin{aligned}
P( \lambda_1, \lambda_2, \tau | \mathbf{y} ) &\propto \left[\prod_{t=1851}^{\tau} \text{Poi}(y_t|\lambda_1) \prod_{t=\tau+1}^{1962} \text{Poi}(y_t|\lambda_2) \right] \text{Gamma}(\lambda_1|\alpha,\beta) \text{Gamma}(\lambda_2|\alpha, \beta) \frac{1}{111} \\
&\propto \left[\prod_{t=1851}^{\tau} e^{-\lambda_1}\lambda_1^{y_t} \prod_{t=\tau+1}^{1962} e^{-\lambda_2} \lambda_2^{y_t} \right] \lambda_1^{\alpha-1} e^{-\beta\lambda_1} \lambda_2^{\alpha-1} e^{-\beta\lambda_2} \\
&\propto \lambda_1^{\sum_{t=1851}^{\tau} y_t +\alpha-1} e^{-(\beta+\tau)\lambda_1} \lambda_2^{\sum_{t=\tau+1}^{1962} y_i + \alpha-1} e^{-\beta\lambda_2}
\end{aligned}$$

So, the full conditionals are known, and critically for Gibbs, can easily be sampled from.

$$\lambda_1 \sim \text{Gamma}(\sum_{t=1851}^{\tau} y_t +\alpha, \tau+\beta)$$
$$\lambda_2 \sim \text{Gamma}(\sum_{t=\tau+1}^{1962} y_i + \alpha, 1962-\tau+\beta)$$
$$\tau \sim \text{Categorical}\left( \frac{\lambda_1^{\sum_{t=1851}^{\tau} y_t +\alpha-1} e^{-(\beta+\tau)\lambda_1} \lambda_2^{\sum_{t=\tau+1}^{1962} y_i + \alpha-1} e^{-\beta\lambda_2}}{\sum_{k=1851}^{1962} \lambda_1^{\sum_{t=1851}^{\tau} y_t +\alpha-1} e^{-(\beta+\tau)\lambda_1} \lambda_2^{\sum_{t=\tau+1}^{1962} y_i + \alpha-1} e^{-\beta\lambda_2}} \right)$$

In [ ]:
def gamma_pdf(lam, a, b):
    return lam**(a - 1) * np.exp(-b * lam)


def gibbs_sample_disaster(samples, tau=1900, early_lambda=6, late_lambda=2):
    """Can supply different initial conditions!"""
    draws = np.empty((3, samples))
    n_years = disasters_array.shape[0]
    years = np.arange(1851, 1962, dtype=int)
    # draws = []
    # while len(draws) < samples:
    for i in range(samples):
        # update early_lambda
        early_lambda = np.random.gamma(disasters_array[:tau - 1851].sum() + 1, 1 / (tau - 1851 + 10))
        # draws.append([early_lambda, late_lambda, tau])
        draws[0, i] = early_lambda
        
        # update late_lambda
        late_lambda = np.random.gamma(disasters_array[tau - 1851 + 1:].sum() + 1, 1 / (1962 - tau + 10))
        # draws.append([early_lambda, late_lambda, tau])
        draws[1, i] = late_lambda
        
        # update tau
        tau_probs = np.empty(n_years)
        for t in range(n_years):
            tau_probs[t] = (gamma_pdf(early_lambda, disasters_array[:t].sum() + 1, t + 10) *
                            gamma_pdf(late_lambda, disasters_array[t:].sum() + 1, n_years - t + 10))
        tau = np.random.choice(years, p=tau_probs / tau_probs.sum())
        # draws.append([early_lambda, late_lambda, tau])
        draws[2, i] = tau

    return draws #np.array(draws)[:samples]

Now we can draw samples using our custom Gibbs sampler:

In [ ]:
draws = gibbs_sample_disaster(1000)
draws.mean(axis=1) # early_lambda, late_lambda, change_point

We can compare the results of our Gibbs sampler to PyMC's built-in samplers:

In [ ]:
with coal_disaster_model() as model:
    trace = pm.sample()

In [ ]:
az.summary(trace, var_names=['early_lambda', 'late_lambda', 'change_point'])

## Hamiltonian Monte Carlo

While flexible and easy to implement, Metropolis-Hastings (and Gibbs) sampling is a random walk
sampler that might not be statistically efficient for many models. Specifically, for models of high dimension, random walk jumping algorithms do not perform well. It is not enough to simply guess at the next sample location; we need to make each iteration a useful draw from the posterior whenever we can, in order to have an efficient sampler for bigger models.

Since Bayesian inference is all about calculating expectations over posteriors, what we seek is an algorithm that explores the area of the parameter space that contains most of the non-zero probability. This region is called the **typical set**.

### What's a Typical Set?

The typical set is where most of the probability density (mass) lies in a particular volume associated with the distribution. As the dimension of a model increases, this set moves progressively further from the mode, and becomes more singular, as the result of concentration of measure.

The typical set is a product of both the density, which is highest at the mode, and volume (that we integrate over), which increasingly becomes larger away from the mode as dimensionality increases. In fact, at high dimensions, the region around the mode contributes almost nothing to the expectation. We need an algorithm that will find this narrow region and explore it efficiently.

![from Hoffman and Gelman 2014](http://d.pr/i/RAA+)

In this context, and when sampling from continuous variables, Hamiltonian (or Hybrid) Monte
Carlo (HMC) can prove to be a powerful tool. It avoids
random walk behavior by simulating a physical system governed by
Hamiltonian dynamics, potentially avoiding tricky conditional
distributions in the process.

In HMC, model samples are obtained by simulating a physical system,
where particles move about a high-dimensional landscape, subject to
potential and kinetic energies. Adapting the notation from [Neal (1993)](http://www.cs.toronto.edu/~radford/review.abstract.html),
particles are characterized by a position vector or state
$s \in \mathcal{R}^D$ and velocity vector $\phi \in \mathcal{R}^D$. The
combined state of a particle is denoted as $\chi=(s,\phi)$. 

The joint **canonical distribution** of the position and velocity can be expressed as a product of the marginal position (which is of interest) and the conditional distribution of the velocity:

$$\pi(s, \phi) = \pi(\phi | s) \pi(s)$$

This joint probability can also be written in terms of an invariant **Hamiltonian function**:

$$\pi(s, \phi) \propto \exp(-H(s,\phi))$$

The Hamiltonian is then defined as the sum of potential energy $E(s)$ and kinetic energy
$K(\phi)$, as follows:

$$\mathcal{H}(s,\phi) = E(s) + K(\phi)
= E(s) + \frac{1}{2} \sum_i \phi_i^2$$

Instead of sampling $p(s)$ directly, HMC operates by sampling from the canonical distribution.

$$p(s,\phi) = \frac{1}{Z} \exp(-\mathcal{H}(s,\phi))=p(s)p(\phi)$$

If we choose a momentum that is independent of position, marginalizing over $\phi$ is
trivial and recovers the original distribution of interest.

Note that the Hamiltonian $\mathcal{H}$ is independent of the parameterization of the model, and therefore, captures the geometry of the phase space distribution, including typical set. 

**Hamiltonian Dynamics**

State $s$ and velocity $\phi$ are modified such that
$\mathcal{H}(s,\phi)$ remains constant throughout the simulation. The
differential equations are given by:

$$\begin{aligned}\frac{ds_i}{dt} &= \frac{\partial \mathcal{H}}{\partial \phi_i} = \phi_i \\
\frac{d\phi_i}{dt} &= - \frac{\partial \mathcal{H}}{\partial s_i}
= - \frac{\partial E}{\partial s_i}
\end{aligned}$$

As shown in [Neal (1993)](http://www.cs.toronto.edu/~radford/review.abstract.html), 
the above transformation preserves volume and is
reversible. The above dynamics can thus be used as transition operators
of a Markov chain and will leave $p(s,\phi)$ invariant. That chain by
itself is not ergodic however, since simulating the dynamics maintains a
fixed Hamiltonian $\mathcal{H}(s,\phi)$. HMC thus alternates Hamiltonian
dynamic steps, with Gibbs sampling of the velocity. Because $p(s)$ and
$p(\phi)$ are independent, sampling $\phi_{new} \sim p(\phi|s)$ is
trivial since $p(\phi|s)=p(\phi)$, where $p(\phi)$ is often taken to be
the univariate Gaussian.

![Skate park](images/skate_park.png?raw=true)

**The Leap-Frog Algorithm**

In practice, we cannot simulate Hamiltonian dynamics exactly because of
the problem of time discretization. There are several ways one can do
this. To maintain invariance of the Markov chain however, care must be
taken to preserve the properties of *volume conservation* and *time
reversibility*. The **leap-frog algorithm** maintains these properties
and operates in 3 steps:

$$\begin{aligned}
\phi_i(t + \epsilon/2) &= \phi_i(t) - \frac{\epsilon}{2} \frac{\partial{}}{\partial s_i} E(s(t)) \\
s_i(t + \epsilon) &= s_i(t) + \epsilon \phi_i(t + \epsilon/2) \\
\phi_i(t + \epsilon) &= \phi_i(t + \epsilon/2) - \frac{\epsilon}{2} \frac{\partial{}}{\partial s_i} E(s(t + \epsilon)) 
\end{aligned}$$

We thus perform a half-step update of the velocity at time
$t+\epsilon/2$, which is then used to compute $s(t + \epsilon)$ and
$\phi(t + \epsilon)$.

**Accept / Reject**

In practice, using finite stepsizes $\epsilon$ will not preserve
$\mathcal{H}(s,\phi)$ exactly and will introduce bias in the simulation.
Also, rounding errors due to the use of floating point numbers means
that the above transformation will not be perfectly reversible.

HMC cancels these effects **exactly** by adding a Metropolis
accept/reject stage, after $n$ leapfrog steps. The new state
$\chi' = (s',\phi')$ is accepted with probability $p_{acc}(\chi,\chi')$,
defined as:

$$p_{acc}(\chi,\chi') = min \left( 1, \frac{\exp(-\mathcal{H}(s',\phi')}{\exp(-\mathcal{H}(s,\phi)} \right)$$

**HMC Algorithm**

We obtain a new HMC sample as follows:

1.  sample a new velocity from a univariate Gaussian distribution
2.  perform $n$ leapfrog steps to obtain the new state $\chi'$
3.  perform accept/reject move of $\chi'$

## No U-Turn Sampler (NUTS)

The No U-Turn Sampler (NUTS) is an extension of HMC that automatically tunes the number of leapfrog steps and step size parameters for optimal performance.

In standard HMC, we need to specify both a step size and a number of steps. The step size needs to be small enough to maintain a reasonable acceptance rate, but large enough to make progress. The number of steps needs to be large enough to explore the posterior efficiently, but not so large that computation time becomes excessive.

NUTS addresses this by:

1. Adaptively selecting step sizes during warmup to achieve a target acceptance rate
2. Dynamically determining the appropriate number of steps by monitoring the trajectory's progress

The "No U-Turn" part of the name comes from the key insight: the algorithm stops taking steps when the trajectory starts to double back on itself (making a "U-turn"), as this indicates it's no longer efficiently exploring new parts of the posterior distribution.

## MCMC in PyMC

PyMC's core business is using Markov chain Monte Carlo to fit virtually any probability model. This involves the assignment and coordination of a suite of **step methods**, each of which is responsible for updating one or more variables. 

The user's interface to PyMC's sampling algorithms is the `sample` function, which coordinates the sampling process for a model:

In [ ]:
with coal_disaster_model() as model:
    # Sample from the posterior
    trace = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED)

### Supported sampling methods

PyMC offers a range of sampling methods to accommodate different model requirements:

1. **NUTS (No U-Turn Sampler)** - The default for continuous variables, an advanced HMC variant that automatically tunes parameters

2. **HamiltonianMC** - Basic Hamiltonian Monte Carlo implementation for continuous variables

3. **Metropolis** - Classic Metropolis-Hastings algorithm that can handle continuous and discrete variables

4. **BinaryMetropolis** - Specialized algorithm for binary (boolean) variables

5. **Slice** - Slice sampling algorithm

6. **SGMC (Stochastic Gradient MCMC)** - Methods that use gradient information for large datasets

The `sample` function generally selects the appropriate method automatically based on the variable types in your model, though you can manually specify methods if needed.

### Parallel sampling

PyMC makes it easy to run multiple chains in parallel, which can utilize multi-core processors and provide more robust inference:

In [ ]:
with coal_disaster_model() as model:
    # Sample 4 chains in parallel using 4 cores
    trace = pm.sample(1000, tune=1000, chains=4, cores=4)

### JAX-based Samplers

PyMC also offers high-performance JAX-based samplers. These leverage the JAX framework for faster computation, especially on GPUs and TPUs. You can use them by specifying the `nuts_sampler` parameter:

In [ ]:
with coal_disaster_model() as model:
    # Use the Nutpie JAX sampler
    trace = pm.sample(nuts_sampler="nutpie", nuts_sampler_kwargs={"backend": "jax"})

## Exercise: Coal mining disasters

Return to the coal mining disasters example. Let's try implementing it using different samplers and comparing performance:

In [ ]:
# Try implementing with different samplers and compare
# You can use Metropolis, NUTS, and/or JAX-based samplers

---

## References

1. Ching & Chen. 2007. Transitional Markov chain Monte Carlo method for Bayesian model updating, model class selection and model averaging. Journal of Engineering Mechanics 2007
2. Hoffman MD, Gelman A. 2014. The No-U-turn sampler: adaptively setting path lengths in Hamiltonian Monte Carlo. The Journal of Machine Learning Research. 15(1):1593-1623.
3. M.I. Jordan. 2004. Graphical models. Statist. Sci., 19(1):140–155.
4. Neal, R. M. 2003. Slice sampling. The Annals of Statistics, 31(3), 705–767. doi:10.1111/1467-9868.00198
5. Gelman, A., Carlin, J. B., Stern, H. S., & Rubin, D. B. (2003). Bayesian Data Analysis, Second Edition (Chapman & Hall/CRC Texts in Statistical Science) (2nd ed.). Chapman and Hall/CRC.

In [ ]:
import watermark
%watermark -n -u -v -iv -w